<a href="https://colab.research.google.com/github/ronaldocr9803/Natural-Language-Processing-in-Tensorflow/blob/master/Course_3_Week_2_Lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-datasets

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews",with_info = True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7N9RF5/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7N9RF5/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7N9RF5/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_senetences = []
training_labels = []

testing_sentences = []
testing_labels = []


In [5]:
for s,l in train_data:
  training_senetences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in train_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [6]:
#1:positive, 0:negative
#when training, labels are expected to be numpy arrays
import numpy as np
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)



In [7]:
#Tokenize the sentences
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_senetences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_senetences)
padded = pad_sequences(sequences,maxlen = max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen = max_length)


In [17]:
print(len(testing_padded))

25000


In [8]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')

])

In [12]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4977 - accuracy: 0.7362 - val_loss: 0.3188 - val_accuracy: 0.8631
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2411 - accuracy: 0.9074 - val_loss: 0.2857 - val_accuracy: 0.8806
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0923 - accuracy: 0.9770 - val_loss: 0.3252 - val_accuracy: 0.8738
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0239 - accuracy: 0.9969 - val_loss: 0.3705 - val_accuracy: 0.8765
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0057 - accuracy: 0.9995 - val_loss: 0.4133 - val_accuracy: 0.8774
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.4457 - val_accuracy: 0.8778
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.5278e-04 - accuracy: 1.0000 - val_loss: 0.4755 - val_accuracy: 0.8777
Ep